In [10]:
# Imports
!pip install -q transformers[torch] datasets pysentimiento accelerate evaluate
from datasets import load_dataset, load_dataset_builder, get_dataset_split_names, load_dataset, concatenate_datasets, DatasetDict
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, pipeline
from pysentimiento import create_analyzer
from pysentimiento.preprocessing import preprocess_tweet
import random
import torch
import re

In [2]:
# Eliminar saltos de línea y espacios repetidos
def delete_spaces(comment):
    spaces_pattern = r'[\n\r]+|\s+'
    return re.sub(spaces_pattern, ' ', comment)

# Convertir todo a minúsculas
def lower_text(comment):
    return comment.lower()

# Eliminar URL
def delete_urls(comment):
    url_pattern = r'http[s]?://\S+'
    return re.sub(url_pattern, '', comment)

# Eliminar consonantes repetidas y puntos suspensivos
def delete_repeated_consonants(comment):
    repeated_consonant_pattern = r'([^aeiou\s\r\n0-9])\1{1,}'
    def replace(match):
        char = match.group(1)
        if char in 'rcnl':
            return char * 2
        else:
            return char

    return re.sub(repeated_consonant_pattern, replace, comment, flags=re.IGNORECASE)

# Dejar como máximo 2 vocales iguales contiguas
def delete_repeated_vowels(comment):
    repeated_vowels_pattern = r'([aeiouAEIOU])\1{2,}'
    return re.sub(repeated_vowels_pattern, r'\1\1', comment, flags=re.IGNORECASE)

# Eliminar acentos no empleados en Español
def delete_accents (comment):
    comment = re.sub(r"[àâãäå]", "a", comment)
    comment = re.sub(r"ç", "c", comment)
    comment = re.sub(r"[èêë]", "e", comment)
    comment = re.sub(r"[ìîï]", "i", comment)
    comment = re.sub(r"[òôõö]", "o", comment)
    comment = re.sub(r"[ùû]", "u", comment)
    comment = re.sub(r"[ýÿ]", "y", comment)
    return comment

# Eliminar caracteres inusuales
def delete_characters(comment):
    special_characters = r'[ºª|·~¬\^`[\]¨´#\\\'\(\)*\<>_]'
    return re.sub(special_characters, '', comment)

# Eliminar emoticonos
def delete_emoticons(comment):
    emoticon_pattern = r'[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F900-\U0001F9FF]'
    return re.sub(emoticon_pattern, '', comment)

# Unificar las distintas formas de expresar la risa
def unify_laughs (comment):
    laugh_pattern = r"\b(a*ha+h[ha]*|o?l+o+l+[ol]*|x+d+[x*d*]*|a*ja+[j+a+]+|j+e+j+[ej]*)\b"
    return re.sub(laugh_pattern, 'jaja', comment, flags=re.IGNORECASE)

# Función para preprocesar el texto
def preprocess_comment(comment):
    comment = delete_spaces(comment)
    comment = lower_text(comment)
    comment = delete_urls(comment)
    comment = delete_repeated_consonants(comment)
    comment = delete_repeated_vowels(comment)
    comment = delete_accents(comment)
    comment = delete_characters(comment)
    comment = delete_emoticons(comment)
    comment = unify_laughs(comment)
    return comment

In [3]:
# Cargar el dataset
database_checkpoint = "amaiaruvi/news_racist_comments_spanish"
dataset = load_dataset(database_checkpoint)
dataset

Generating train split:   0%|          | 0/3005 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/438 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/851 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['link', 'title', 'comment', 'racist'],
        num_rows: 3005
    })
    validation: Dataset({
        features: ['link', 'title', 'comment', 'racist'],
        num_rows: 438
    })
    test: Dataset({
        features: ['link', 'title', 'comment', 'racist'],
        num_rows: 851
    })
})

In [4]:
# Cargar el modelo
modelo = "pysentimiento/robertuito-hate-speech"
tokenizer = AutoTokenizer.from_pretrained(modelo)
model = AutoModelForSequenceClassification.from_pretrained(modelo)

tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/956 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/435M [00:00<?, ?B/s]

In [5]:
# Configuración del modelo:
model.config

RobertaConfig {
  "_name_or_path": "pysentimiento/robertuito-hate-speech",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "hateful",
    "1": "targeted",
    "2": "aggressive"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "aggressive": 2,
    "hateful": 0,
    "targeted": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 130,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "multi_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.41.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 30002
}

In [6]:
tokenizer.all_special_tokens

['<s>', '</s>', '<unk>', '<pad>', '<mask>']

In [7]:
# Este modelo sólo admite 128 tokens por cada sentencia; por lo que hay que truncar
tokenizer.model_max_length

128

In [8]:
tokenizer.get_vocab()

{'▁plat': 19093,
 '▁zorra': 21280,
 '▁glorioso': 27944,
 '▁tanque': 19715,
 '▁ayudarme': 23034,
 'indus': 29788,
 '▁lau': 9978,
 '▁insign': 22235,
 '▁vence': 13896,
 'ksj': 22370,
 '▁sanidad': 7982,
 '▁demente': 29262,
 '▁afrontar': 17032,
 '▁lmao': 11129,
 '▁obligaciones': 27849,
 'ez.': 16487,
 '▁between': 12505,
 '▁=)': 27403,
 '▁feministas': 12863,
 '▁taemin': 25759,
 '▁canarias': 11521,
 '▁votar': 3157,
 '▁amaba': 21584,
 '▁carác': 10943,
 '▁l.': 14209,
 '▁semáfor': 27972,
 '▁cobardes': 18291,
 '▁gooolll': 21916,
 '▁hermosa': 2653,
 '▁pusimos': 24649,
 'ú,': 26750,
 '▁lomo': 21386,
 '▁campeones': 9145,
 '▁mtv': 1562,
 '▁apoyando': 8485,
 '▁nue': 880,
 '1,': 8372,
 '▁francés': 11777,
 'さい': 28905,
 '▁educativa': 13912,
 '▁quedado': 6894,
 '▁desma': 11848,
 '▁pandemia,': 11089,
 '▁pensaron': 28821,
 '▁teaser': 15568,
 'el.': 14887,
 '▁belo': 15306,
 '▁tocará': 25075,
 '▁trabajaba': 27234,
 '▁bd': 24794,
 '▁inteli': 3691,
 '▁career': 26859,
 '▁inventan': 25619,
 '▁multic': 29713,
 '▁

In [11]:
# Se especifica que para utilizar el modelo ""pysentimiento/robertuito-hate-speech"
# antes hay que preprocesar el texto con su función "preprocess_tweet".

# Añadimos la separacón entre comentario y título manualmente para poder
# utilizar el tokenizador original en el pipeline. Pipeline no funciona si le
# pasamos dos parámetros al tokenizador en volviéndolo en una función tokenize.

print("Preprocessing data...")
sep_token = tokenizer.sep_token
preprocessed_data = dataset.map(lambda ex: {
    "text": sep_token.join([
        preprocess_comment(preprocess_tweet(ex["comment"], lang="es")),
        preprocess_comment(preprocess_tweet(ex["title"], lang="es"))
    ])
})

Preprocessing data...


Map:   0%|          | 0/3005 [00:00<?, ? examples/s]

Map:   0%|          | 0/438 [00:00<?, ? examples/s]

Map:   0%|          | 0/851 [00:00<?, ? examples/s]

In [12]:
def custom_tokenizer(text, **kwargs):
    return tokenizer(
        text,
        padding=True,
        truncation=True,
        max_length=tokenizer.model_max_length,
        #max_length=1024
        **kwargs
    )

'''
Se puede mofificar el parámetro max_length, pero es importante considerar que
aumentar max_length también puede afectar la precisión del modelo en algunas
tareas, especialmente si el modelo se preentrenó con secuencias más cortas.
Esto se debe a que las secuencias más largas pueden contener más ruido o
información irrelevante, lo que podría dificultar que el modelo aprenda
representaciones significativas.
'''

'\nSe puede mofificar el parámetro max_length, pero es importante considerar que\naumentar max_length también puede afectar la precisión del modelo en algunas\ntareas, especialmente si el modelo se preentrenó con secuencias más cortas.\nEsto se debe a que las secuencias más largas pueden contener más ruido o\ninformación irrelevante, lo que podría dificultar que el modelo aprenda\nrepresentaciones significativas.\n'

In [13]:
preprocessed_data['test'][1]

{'link': 'https://okdiario.com/espana/vox-empapela-gerona-carteles-arabe-estas-espana-hombres-mujeres-tienen-mismos-derechos-12797483',
 'title': 'Vox empapela Gerona con carteles en árabe: «Estás en España, hombres y mujeres tienen los mismos derechos»',
 'comment': 'Mira quien habla, los de los tiros en la nuca.',
 'racist': 0,
 'text': 'mira quien habla, los de los tiros en la nuca.</s>vox empapela gerona con carteles en árabe: "estás en españa, hombres y mujeres tienen los mismos derechos"'}

In [14]:
tokenized = custom_tokenizer(preprocessed_data['test'][1]['text'])
tokens_strings = tokenizer.convert_ids_to_tokens(tokenized['input_ids'])

print("Texto a tokenizar:", preprocessed_data['test'][1]['comment'], ' + ', preprocessed_data['test'][1]['title'])
print("Tokens:", tokens_strings)
print("\n\ninput_ids:", tokenized['input_ids'])
print("token_type_ids:", tokenized['token_type_ids'])
print("attention_mask:", tokenized['attention_mask'])

Texto a tokenizar: Mira quien habla, los de los tiros en la nuca.  +  Vox empapela Gerona con carteles en árabe: «Estás en España, hombres y mujeres tienen los mismos derechos»
Tokens: ['<s>', '▁mira', '▁quien', '▁habla,', '▁los', '▁de', '▁los', '▁tiros', '▁en', '▁la', '▁nu', 'ca.', '</s>', '▁vox', '▁empa', 'pe', 'la', '▁ger', 'ona', '▁con', '▁carteles', '▁en', '▁ára', 'be', ':', '▁"', 'est', 'ás', '▁en', '▁españa,', '▁hombres', '▁y', '▁mujeres', '▁tienen', '▁los', '▁mismos', '▁derechos', '"', '</s>']


input_ids: [0, 1659, 1005, 24880, 497, 413, 497, 13470, 452, 446, 2255, 4531, 2, 4601, 6103, 724, 486, 4387, 918, 461, 21312, 452, 17837, 773, 30, 576, 2005, 537, 452, 8478, 2976, 445, 2054, 1215, 497, 4212, 3186, 6, 2]
token_type_ids: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
attention_mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [15]:
# OPCIÓN 1: Utilizar la función Pipeline de Hugging Face
pipe = pipeline('text-classification', model=modelo, tokenizer=custom_tokenizer)
pipeline_predictions = []
for i, comment in enumerate(preprocessed_data['test']):
    result = pipe(comment['text'])
    pipeline_predictions.append(result)
print("Pipeline: Output of the first prediction:", pipeline_predictions[0])

Pipeline: Output of the first prediction: [{'label': 'hateful', 'score': 0.542149007320404}]


In [16]:
# OPCIÓN 2: Utilizar la función create_analyzer de la librería pysentimiento
# especificando la tarea "hate_speech", ya que carga el mismo modelo.

# Los resultados son los mismos, pero desde aquí te sacan "output" si la probabilidad
# es mayor de 0,5 y además te devuelve las probabilidades de "targeted" y "aggresive".
hate_speech_analyzer = create_analyzer(task="hate_speech", lang="es")
analyzer_predictions = hate_speech_analyzer.predict(preprocessed_data['test']['text'])
print("Analyzer: Output of the first prediction:", analyzer_predictions[0])

Map:   0%|          | 0/851 [00:00<?, ? examples/s]

Analyzer: Output of the first prediction: AnalyzerOutput(output=['hateful'], probas={hateful: 0.542, targeted: 0.019, aggressive: 0.169})


In [17]:
# Extraer las etiquetas reales y las predicciones
etiquetas_reales = preprocessed_data['test']['racist']
etiquetas_predichas = [pred.output for pred in analyzer_predictions]
etiquetas_predichas_transformadas = [1 if 'hateful' in elemento else 0 for elemento in etiquetas_predichas]

In [18]:
# Calcular las métricas de rendimiento
print("Calculating metrics...")
reporte = classification_report(etiquetas_reales, etiquetas_predichas_transformadas, output_dict=False)
print(reporte)

Calculating metrics...
              precision    recall  f1-score   support

           0       0.86      0.81      0.83       654
           1       0.47      0.55      0.51       197

    accuracy                           0.75       851
   macro avg       0.66      0.68      0.67       851
weighted avg       0.77      0.75      0.76       851

